# Run models and create submissions

## I) imports

In [6]:
from proj1_helpers import *
from custom_helpers import *
from implementations import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## II) Required functions

In [27]:
def train_model(y_train, x_train, flag_method, max_iter=1000, gamma=1e-6, lambda_=0): 
    """Train a model on a given subset of the data. Choose method by setting flag_method"""

    initial_w = np.ones(x_train.shape[1])

    if flag_method == 0:
        # Use linear regression (full gradient descent)
        weight, loss_tr = least_squares_GD(y_train, x_train, initial_w, max_iters, gamma)
            
    if flag_method == 1:
        # Use linear regression (stochastic gradient descent)
        weight, loss_tr = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)
        
    if flag_method == 2:
        # Use least squares method
        weight, loss_tr = least_squares(y_train, x_train)
            
    if flag_method == 3:
        # Use ridge regression
        weight, loss_tr = ridge_regression(y_train, x_train, lambda_)
           
    if flag_method == 4:
        # Use logistic regression
        weight, loss_tr = logistic_regression(y_train, x_train, initial_w, max_iters, gamma)
            
    if flag_method == 5:
        # Use regularized logistic regression
        weight, loss_tr = reg_logistic_regression(y_train, x_train, initial_w, max_iters, gamma, lambda_)
        
    return weight

## III) Load data

In [8]:
# Load Data
print("Loading Data, please wait")
y_test, x_test_raw, ids_test = load_csv_data('data/test.csv')
y_train, x_train_raw, ids_train = load_csv_data('data/train.csv')
print("Data loaded, continue!!")

Loading Data, please wait
Data loaded, continue!!


## IV) Train Model on the training set and make predictions on the testset
Chose from the following methods:

In [4]:
"""
Methods mapping
0    Linear regression (full gradient descent)
1    Linear regression (stochastic gradient descent)
2    Least squares method
3    Ridge regression
4    Logistic regression (stochastic gradient descent)
5    Regularized logistic regression (stochastic gradient descent)
"""

'\nMethods mapping\n0    Linear regression (full gradient descent)\n1    Linear regression (stochastic gradient descent)\n2    Least squares method\n3    Ridge regression\n4    Logistic regression (stochastic gradient descent)\n5    Regularized logistic regression (stochastic gradient descent)\n'

### Train one model only

In [28]:
# Choose feature treatment methods
flag_add_offset = True
flag_standardize = True
flag_remove_outliers = True
degree = 1

# Choose training model to apply (see mapping above)
flag_method = 4

# Set training parameters
max_iters = 20000
gamma = 0.000001
lambda_ = 0.0

# Prepare data
print("Preparing data...")
x_train, x_test = prepare_data(x_train_raw, x_test_raw, flag_add_offset, flag_standardize, flag_remove_outliers, degree)

# Train model
print("Training starts...")
weight = train_model(y_train, x_train, flag_method, max_iters, gamma, lambda_)
print("Training done!")

# Make predictions
pred_y = predict_labels(weight, x_test)
ids_pred_y = ids_test
print("Predictions ready. You can now write them to a file")

Preparing data...
Training starts...
iteration	 0 	loss:  0.014062540072781743


KeyboardInterrupt: 

### Train a seperate model for each of the jets

In [19]:
# Choose feature treatment methods
flag_add_offset = True
flag_standardize = True
flag_remove_outliers = True
degree = 1

# Choose training model to apply (see mapping above)
flag_method = 5

# Set training parameters
max_iters = 2000
gamma = 0.000001
lambda_ = 0.0


# In the dateset, we found that the Column[22] PRI_jet_num dataset is categorical with Four categories defined.column_jet_nb = 22
pred_y = []
ids_pred_y = []
column_categorical=22
# In the paper where describes the differente features of the data, explains that different columns are invalid values
# depending on the value of the categorical feature, so we can delete those values for the 4 different trainings
#The undefined features, with first vector for the categorical value of 0, and so on.
undefined_features = [[4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28, 29], [4, 5, 6, 12, 22, 26, 27, 28], [22], [22]]

#We will have a for loop with 4 values for the 4 categorical training
for nb_jets in range(0, 4):
    print("Cleaning and preparing data for jet number %d" %nb_jets)
    # We will separate select data according to the value of the categorical values for each loop in our cicle
    jet_index_test = x_test_raw[:, column_categorical] == nb_jets
    x_test_jet = x_test_raw[jet_index_test]
    y_test_jet = y_test[jet_index_test]
    id_test_jet = ids_test[jet_index_test]
    jet_index_train = x_train_raw[:, column_categorical] == nb_jets
    x_train_jet = x_train_raw[jet_index_train]
    y_train_jet = y_train[jet_index_train]
    id_train_jet = ids_train[jet_index_train]
    
    #remove undefined features
    x_test_jet = np.delete(x_test_jet, undefined_features[nb_jets], axis=1)
    x_train_jet = np.delete(x_train_jet, undefined_features[nb_jets], axis=1)
       
    # Prepare data
    x_train_jet, x_test_jet = prepare_data(x_train_jet, x_test_jet, flag_add_offset, flag_standardize, flag_remove_outliers, degree)

    print("Training model for jet number %d..." %nb_jets)
    # train the chosen model
    weight = train_model(y_train_jet, x_train_jet, flag_method, max_iters, gamma, lambda_)
    
    print("making predictions for jet number %d..." %nb_jets)
    # Now we get the prediction for y
    pred_y_jet = predict_labels(weight, x_test_jet)
    pred_y.extend(pred_y_jet)
    ids_pred_y.extend(id_test_jet) 

print("Finished training all four models. Predictions are ready to be written to file")

Cleaning and preparing data for jet number 0
Training model for jet number 0...
iteration	 0 	loss:  0.015440748667591288
iteration	 100 	loss:  0.7049122758667792
iteration	 200 	loss:  0.49611141473711134
iteration	 300 	loss:  0.44396185908359265
iteration	 400 	loss:  0.42735839887454014
iteration	 500 	loss:  0.42086334610093135
iteration	 600 	loss:  0.4179044449988276
iteration	 700 	loss:  0.4163815798431014
iteration	 800 	loss:  0.41550661987418563
iteration	 900 	loss:  0.41494787325533095
iteration	 1000 	loss:  0.41455406960672225


KeyboardInterrupt: 

## V) Create submission file

In [44]:
# Choose filename:
filename = "submissionX.csv"

print("Creating submission file.")
create_csv_submission(ids_pred_y, pred_y, filename)
print("Created submission file!")

Creating submission file.
Created submission file!


In [20]:
"""
TODO:   fine-tune hyperparameters to make better submission
        Run with less feature engineering to prove that our choices improve the result


"""

'\nTODO:   fine-tune hyperparameters to make better submission\n        Run with less feature engineering to prove that our choices improve the result\n\n\n'